# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
import re
searchre = re.compile(r"Babelgum")
cur = db.companies.find({"name":searchre})
for c in cur:
    print(c["name"])



Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [3]:
from pymongo import ASCENDING, DESCENDING
from pandas.io.json import json_normalize
rest=list(db.companies.find({"number_of_employees":{"$gt":5000}},{"name":1,"number_of_employees":1,'_id':0}).sort([
   ("number_of_employees", DESCENDING)
]).limit(20))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [16]:
qsearch={"$and":
         [{"founded_year":{"$gte":2000}},
          {"founded_year":{"$lte":2005}}]
        }
rest=list(db.companies.find(qsearch,  {"name":1,
          "founded_year": 1,
          "_id":0 }).sort([("number_of_employees", DESCENDING)
]))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,founded_year
0,Accenture,2001
1,MetaCarta,2001
2,Avaya,2000
3,Webkinz,2005
4,Liberty League International,2001
...,...,...
3729,Axon Solutions,2004
3730,EnterSys Group,2000
3731,Intergy,2003
3732,AfterLogic,2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [17]:
qsearch={"$and":
         [{"ipo.valuation_amount":{"$gt":100000000}},
          {"founded_year":{"$lt":2010}}]
        }
rest=list(db.companies.find(qsearch,  {"name":1,
          "ipo": 1,
          "_id":0 }).sort([("ipo", DESCENDING)
]))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,ipo.valuation_amount,ipo.valuation_currency_code,ipo.pub_year,ipo.pub_month,ipo.pub_day,ipo.stock_symbol
0,GREE,108960000000,JPY,2008.0,12.0,17.0,3632
1,Facebook,104000000000,USD,2012.0,5.0,18.0,NASDAQ:FB
2,Amazon,100000000000,USD,1997.0,5.0,NaN,NASDAQ:AMZN
3,Twitter,18100000000,USD,2013.0,11.0,7.0,NYSE:TWTR
4,Groupon,12800000000,USD,2011.0,11.0,7.0,NASDAQ:GRPN
5,Tencent,11000000000,USD,2004.0,6.0,16.0,HK:0700
6,Western Digital,9430000000,USD,NaN,NaN,NaN,NYSE:WDC
7,LinkedIn,9310000000,USD,2011.0,7.0,20.0,NYSE:LNKD
8,BMC Software,6000000000,USD,1988.0,8.0,12.0,NASDAQ:BMC
9,Rackspace,5440000000,USD,2011.0,11.0,7.0,RAX


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [19]:
qsearch={"$and":
         [{"founded_year":{"$lt":2005}},
          {"number_of_employees":{"$lt":1000}}]
        }
rest=list(db.companies.find(qsearch,  {"name":1,
          "number_of_employees": 1,
          "_id":0 }).sort([("number_of_employees", DESCENDING)
]).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,number_of_employees
0,Infinera Corporation,974
1,NorthPoint Communications Group,948
2,888 Holdings,931
3,Forrester Research,903
4,SonicWALL,900
5,Webmetrics,900
6,Cornerstone OnDemand,881
7,Yelp,800
8,MySpace,800
9,ZoomInfo,800


### 6. All the companies that don't include the `partners` field.

In [22]:
qsearch={"partners": { "$exists" : False } }
rest=list(db.companies.find(qsearch,  {"name":1,
                    "_id":0 })
         )
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


""


### 7. All the companies that have a null type of value on the `category_code` field.

In [25]:
qsearch={"category_code": { "$eq" : None } }
rest=list(db.companies.find(qsearch,  {"name":1,
                    "_id":0 })
         )
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing
...,...
2746,Nellix
2747,Cantimer
2748,cruisecritic
2749,Geekdive


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [27]:
qsearch={"$and":
         [{"number_of_employees":{"$gte":100}},
          {"number_of_employees":{"$lt":1000}}]
        }
rest=list(db.companies.find(qsearch,  {"name":1,
          "number_of_employees": 1,
          "_id":0 }).sort([("number_of_employees", DESCENDING)
]))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,number_of_employees
0,Datamonitor,984
1,Infinera Corporation,974
2,Box,950
3,NorthPoint Communications Group,948
4,888 Holdings,931
...,...,...
912,ZoomSystems,100
913,Exent,100
914,Mashable,100
915,Applied Language Solutions,100


### 9. Order all the companies by their IPO price in a descending order.

In [30]:
qsearch={"ipo.valuation_amount": { "$ne" : None } }
rest=list(db.companies.find(qsearch,  {"name":1,
          "ipo": 1,
          "_id":0 }).sort([("ipo", DESCENDING)
]))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,ipo.valuation_amount,ipo.valuation_currency_code,ipo.pub_year,ipo.pub_month,ipo.pub_day,ipo.stock_symbol
0,GREE,108960000000,JPY,2008.0,12.0,17.0,3632
1,Facebook,104000000000,USD,2012.0,5.0,18.0,NASDAQ:FB
2,Amazon,100000000000,USD,1997.0,5.0,NaN,NASDAQ:AMZN
3,Twitter,18100000000,USD,2013.0,11.0,7.0,NYSE:TWTR
4,Groupon,12800000000,USD,2011.0,11.0,7.0,NASDAQ:GRPN
...,...,...,...,...,...,...,...
56,Energy and Power Solutions,25000000,USD,2010.0,3.0,31.0,EPS
57,Vringo,11003200,USD,2010.0,6.0,22.0,VRNG
58,GT Solar,5000000,USD,2008.0,7.0,23.0,SOLR
59,DeNA,117900,JPY,2005.0,2.0,25.0,TYO:2432


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [31]:
qsearch={}
rest=list(db.companies.find(qsearch,  {"name":1,
          "number_of_employees": 1,
          "_id":0 }).sort([("number_of_employees", DESCENDING)
]).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [32]:
qsearch={"founded_month":{"$gt":6}}
rest=list(db.companies.find(qsearch, {"name":1,
          "_id":0 }).limit(1000))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name
0,Wetpaint
1,Zoho
2,Omnidrive
3,Digg
4,Joost
...,...
995,SavvyAuntie
996,AccountMaven
997,Chapatiz
998,YOOWALK


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [33]:
qsearch={"$and":
         [{"founded_year":{"$lt":2000}},
          {"acquisition.price_amount":{"$gt":10000}}]
        }
rest=list(db.companies.find(qsearch,  {"name":1,
          "acquisition.price_amount": 1,
          "_id":0 }))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,acquisition.price_amount
0,Postini,625000000
1,SideStep,180000000
2,Recipezaar,25000000
3,Cyworld,7140000
4,PayPal,1500000000
...,...,...
220,Savvion,49000000
221,Inventa Technologies,30000000
222,Universal Microwave,23200000
223,Advanced Control Components,18780000


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [35]:
qsearch={"acquisition.acquired_year":{"$gt":2010} }

rest=list(db.companies.find(qsearch,  {"name":1,
          "acquisition": 1,
          "_id":0 }).sort("acquisition.price_amount", DESCENDING))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,T-Mobile,3.900000e+10,USD,None,http://techcrunch.com/2011/03/20/in-the-race-f...,"In The Race For More Spectrum, AT&T Is Acquiri...",2011,3.0,20.0,AT&T,at-t
1,Goodrich Corporation,1.840000e+10,USD,None,http://www.masshightech.com/stories/2011/09/19...,UTC shells out $18.4 billion for Goodrich,2011,9.0,22.0,United Technologies,united-technologies
2,LSI,6.600000e+09,USD,cash,http://dealbook.nytimes.com/2013/12/16/avago-t...,Avago to Buy LSI for $6.6 Billion,2013,12.0,16.0,Avago Technologies,avago-technologies
3,National Semiconductor,6.500000e+09,USD,cash,"http://www.national.com/news/item/0,1735,1522,...",TI to acquire National Semiconductor Complemen...,2011,4.0,4.0,Texas Instruments,texas-instruments
4,Ariba,4.300000e+09,USD,None,http://techcrunch.com/2012/05/22/sap-to-acquir...,SAP To Acquire Ariba For $4.3 Billion,2012,5.0,22.0,SAP,sap
...,...,...,...,...,...,...,...,...,...,...,...
731,MediaPal,NaN,USD,None,http://www.freshnews.com/news/525524/voxpop-wo...,VoxPop Worldwide Acquires Global Rights to eco...,2011,7.0,25.0,VoxPop Worldwide,voxpop-worldwide
732,Vertro,NaN,USD,stock,http://techcrunch.com/2011/10/17/digital-media...,Digital Media Companies Inuvo And Vertro To Merge,2012,3.0,1.0,Inuvo,inuvo
733,ALOT,NaN,USD,stock,http://techcrunch.com/2011/10/17/digital-media...,Digital Media Companies Inuvo And Vertro To Merge,2012,3.0,1.0,Inuvo,inuvo
734,MyChances,NaN,USD,None,http://www.parchment.com/press-releases/new-st...,New Student-Developed Site Helps College Hopef...,2011,4.0,NaN,Parchment,parchment


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [38]:
qsearch={"founded_year":{"$ne":None}} # he limitado sino me salia error

rest=list(db.companies.find(qsearch,  {"name":1,
          "founded_year": 1,
          "_id":0 }).sort("founded_year", ASCENDING).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,founded_year
0,Alstrasoft,1800
1,SmallWorlds,1800
2,US Army,1800
3,DuPont,1802
4,McKesson,1833
5,Bachmann Industries,1833
6,Bertelsmann,1835
7,Accuity,1836
8,CENTRA,1839
9,WeGame,1840


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [40]:
qsearch={"founded_day":{"$lte":7}} 

rest=list(db.companies.find(qsearch,  {"name":1,
          "founded_year": 1,
          "founded_day": 1,
          "acquisition.price_amount" : 1,
          "_id":0 }).sort("acquisition.price_amount", DESCENDING).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,founded_year,founded_day,acquisition.price_amount
0,Netscape,1994,4,4200000000
1,PayPal,1998,1,1500000000
2,Zappos,1999,1,1200000000
3,Alibaba,1999,1,1000000000
4,Postini,1999,2,625000000
5,Danger,1999,1,500000000
6,Clearwell Systems,2004,6,410000000
7,PrimeSense,2005,1,345000000
8,Amobee,2005,1,321000000
9,BlueLithium,2004,1,300000000


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [44]:
qsearch={ "$and" : [ { "category_code" : { "$eq" : "web" }} ,
                    { "number_of_employees": {"$gt" : 4000 }}
                   ]}

rest=list(db.companies.find(qsearch,  {"name":1,
          "founded_year": 1,
          "number_of_employees" : 1,
          "_id":0 }).sort("number_of_employees", ASCENDING).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,number_of_employees,founded_year
0,Expedia,4400,1996
1,AOL,8000,1985
2,Webkinz,8657,2005
3,Rakuten,10000,1997
4,Los Angeles Times Media Group,10000,1881
5,Groupon,10000,2008
6,Yahoo!,13600,1994
7,eBay,15000,1995
8,Experian,15500,1980


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [46]:
qsearch={ "$and" : [ { "acquisition.price_amount" : { "$gt" : 10000000 }} ,
                    { "acquisition.price_currency_code": {"$eq" : "EUR" }}
                   ]}

rest=list(db.companies.find(qsearch,  {"name":1,
          "acquisition.price_amount": 1,
          "acquisition.price_currency_code" : 1,
          "_id":0 }).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,acquisition.price_amount,acquisition.price_currency_code
0,ZYB,31500000,EUR
1,Apertio,140000000,EUR
2,Greenfield Online,40000000,EUR
3,Webedia,70000000,EUR
4,Wayfinder,24000000,EUR
5,Tuenti Technologies,70000000,EUR
6,BioMed Central,43400000,EUR


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [49]:
qsearch={ "acquisition.acquired_month" : { "$lt" : 4 }}
                    
rest=list(db.companies.find(qsearch,  {"name":1,
          "acquisition": 1 ,
          "_id":0 }).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,Kyte,NaN,USD,None,http://techcrunch.com/2011/01/31/exclusive-kit...,"KIT digital Acquires KickApps, Kewego AND Kyte...",2011,1,31.0,KIT digital,kit-digital
1,NetRatings,327000000.0,USD,cash,http://login.vnuemedia.com/hr/login/login_subs...,Nielsen buys rest of NetRatings,2007,2,NaN,Nielsen,nielsen
2,blogTV,NaN,USD,None,http://techcrunch.com/2013/03/13/younow-buys-b...,Live Social Video Network YouNow Acquires Stre...,2013,3,13.0,YouNow,younow
3,Livestream,NaN,USD,cash_and_stock,,company,2008,1,1.0,Adviqo,adviqo
4,iContact,169000000.0,USD,cash_and_stock,http://techcrunch.com/2012/02/28/vocus-buys-em...,Vocus Buys Email Marketing Company iContact Fo...,2012,2,28.0,Vocus,vocus
5,Coghead,NaN,USD,None,http://www.techcrunch.com/2009/02/19/sap-acqui...,SAP Acquires Cogheadâ€™s Technology As It Look...,2009,2,19.0,SAP,sap
6,Dailymotion,168000000.0,USD,None,http://techcrunch.com/2013/02/22/orange-acquir...,Orange Acquired Dailymotion For $168 Million B...,2013,2,22.0,Orange,orange
7,Netvibes,NaN,USD,None,http://www.marketwatch.com/story/dassault-syst...,Dassault Systemes Acquires Netvibes,2012,2,9.0,Dassault Systemes,dassault
8,BabyCenter,NaN,USD,None,http://www.investor.jnj.com/releasedetail.cfm?...,Johnson & Johnson Acquires BabyCenter From eTo...,2001,3,2.0,Johnson & Johnson,johnson-johnson
9,Flickr,NaN,USD,cash_and_stock,http://news.cnet.com/Yahoo-buys-photo-sharing-...,CNET,2005,3,NaN,Yahoo!,yahoo


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [58]:
qsearch={"$and":
         [{"founded_year":{"$gte":2000}},
          {"founded_year":{"$lte":2010}},
          {"acquisition.acquired_year":{"$gte":2011}}]}
                    
rest=list(db.companies.find(qsearch,  {"name":1,
                                       "founded_year" :1,
                                       "acquisition": 1 ,
                                       "_id":0 }).limit(10))
json_normalize(rest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.


,name,founded_year,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,Wetpaint,2005,30000000.0,USD,cash_and_stock,http://allthingsd.com/20131216/viggle-tries-to...,Viggle Tries to Bulk Up Its Social TV Busines...,2013,12,16,Viggle,viggle
1,Geni,2006,NaN,USD,None,http://techcrunch.com/2012/11/28/all-in-the-fa...,MyHeritage acquires Geni and $25M to build fam...,2012,11,28,MyHeritage,myheritage
2,Digg,2004,500000.0,USD,None,http://techcrunch.com/2012/07/12/betaworks-acq...,Betaworks Acquires Digg (TechCrunch),2012,7,12,betaworks,betaworks
3,Kyte,2006,NaN,USD,None,http://techcrunch.com/2011/01/31/exclusive-kit...,"KIT digital Acquires KickApps, Kewego AND Kyte...",2011,1,31,KIT digital,kit-digital
4,Jingle Networks,2005,62500000.0,USD,cash_and_stock,http://www.masshightech.com/stories/2011/04/11...,Jingle Networks bought for up to $62.5M,2011,4,11,Marchex,marchex
5,blogTV,2006,NaN,USD,None,http://techcrunch.com/2013/03/13/younow-buys-b...,Live Social Video Network YouNow Acquires Stre...,2013,3,13,YouNow,younow
6,delicious,2003,NaN,USD,None,http://techcrunch.com/2011/04/27/yahoo-sells-d...,Yahoo Sells Delicious To YouTube Founders,2011,4,27,AVOS Systems,avos
7,Revision3,2005,30000000.0,USD,None,http://allthingsd.com/20120503/discovery-gets-...,"Discovery Gets a Web Video Arm, Courtesy of Re...",2012,5,3,Discovery Communications,discoverycommunications
8,iContact,2003,169000000.0,USD,cash_and_stock,http://techcrunch.com/2012/02/28/vocus-buys-em...,Vocus Buys Email Marketing Company iContact Fo...,2012,2,28,Vocus,vocus
9,Mashery,2006,NaN,USD,None,http://allthingsd.com/20130417/intel-acquires-...,Intel Acquires API Manager Mashery,2013,4,17,Intel,intel


### 20. All the companies that have been 'deadpooled' after the third year.

In [57]:
qsearch={"$and":[{"$where" : "this.deadpooled_year > this.founded_year+3" },{"founded_year":{"$ne":None}}]
        }           
rest=list(db.companies.find(qsearch,  {"name":1,
                                       "founded_year" :1,
                                       "deadpooled_year": 1 ,
                                       "_id":0 }).limit(10))
json_normalize(rest) # limit 10 para que no moleste..

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,name,founded_year,deadpooled_year
0,Babelgum,2007,2013
1,Thoof,2006,2013
2,Wesabe,2005,2010
3,Stickam,2006,2013
4,AllPeers,2004,2008
5,EQO,2006,2012
6,AllofMP3,2000,2007
7,SellABand,2006,2010
8,Zlio,2005,2011
9,Jaiku,2006,2012
